In [2]:
import numpy as np
#import yfinance as yf
import datetime
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf

#prices = yf.download(["AAPL"])["Adj Close"]

def rsi(prices, n=14):
    """Compute the RSI given prices
    
    :param prices: pandas.Series
    :return: rsi
    """
    
    # Calculate the difference between the current and previous close price
    delta = prices.diff()
    
    # Calculate the sum of all positive changes
    gain = delta.where(delta > 0, 0)
    
    # Calculate the sum of all negative changes
    loss = -delta.where(delta < 0, 0)
    
    # Calculate the average gain over the last n periods
    avg_gain = gain.rolling(n).mean()
    
    # Calculate the average loss over the last n periods
    avg_loss = loss.rolling(n).mean()
    
    # Calculate the relative strength
    rs = avg_gain / avg_loss
    
    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

#print(rsi(prices))
#print(btc
# to create grid lines with equal distance
def set_grid(highest,lowest,count):
    grids  = np.linspace(lowest , highest , count)
    return grids

def buy_volume(highest_price_vol, lowest_price_vol , count):
    volumes = np.linspace(highest_price_vol , lowest_price_vol , count)
    return volumes

def check_state(candle,grids):
    highest = candle['High']
    lowest = candle["Low"]
    states = []
    for i in range(len(grids)):
        if((grids[i]>lowest)and(highest>grids[i])):
            states.append(i)
    if(len(states)==0):    
        return -1
    else:
        if(candle["Open"]<candle["Close"]):
            return states
        else:
            return states[::-1]

def find_state(candle,grids):
    highest = candle["High"]
    lowest = candle["Low"]
    st = check_state(candle,grids)
    if(st!=-1):
        return st
    for i in range(len(grids)-1):
        if ((grids[i]<lowest)and(grids[i+1]>highest)):
            return i
    if(grids[0]>lowest):
        return -1
    else:
        return len(grids)-1
    
def set_grid(current_price , gap):
    pos_grids  = np.linspace(current_price + gap/2 , current_price + (gap/2)*(61) , 31)
    neg_grids = np.linspace(current_price - (gap/2)*(61),current_price - gap/2,31)
    grids = np.concatenate([neg_grids,pos_grids])
    return grids



In [28]:
def count_grid(market,t_cost,volumes ,start, end,gap,open_pos_count,interval,total_invest,sl,tp,open_date_limit = 0,data = 0,download = True,print_details = 0,rsi_limit = 100,compound_intrest = False):
    print(type(data))
    net_money = total_invest
    first_invest = total_invest
    trade_details = []
    current_open_count = 0
    candle_gap = []
    last_grid = -1
    current_grid = -1
    #print(btc.shape)
    if(download):
        btc = yf.download(market , start = start, end = end , interval = interval)
    else:
        btc = data.loc[start:end]
    #BTC = pd.concat([btc_history,btc])
    BTC = btc
    btc["rsi"] = rsi(btc["Close"])
    #btc = BTC.loc[btc.index]
    print(btc.columns)
    #print(BTC.columns)
    print(btc.shape)
    #print(btc_history.shape)
    #print(btc_history)
    print(btc.iloc[0])
    open_price = btc["Open"].iloc[0]
    close_price = btc["Close"].iloc[-1]
    current_price = (open_price + close_price)*(1/2)
    #highest = np.max(btc_history["High"]) * high_coef
    #print(type(highest))
    #lowest = np.min(btc_history["Low"]) * low_coef
    grids = set_grid(current_price, gap)
    count = len(grids)
    #st = find_state(btc.iloc[0]["High"],btc.iloc[0]["Low"],grids) + 1
    D = (grids[1] - grids[0])
    #V = total_invest/((count * (count+1))*((highest/2) - (2*count+1)*D/6))
    #volumes = pd.Series(buy_volume(0,count*V,count),index = grids[::-1])
    volume = pd.Series([total_invest/open_pos_count]*open_pos_count,index = [i for i in range(1,open_pos_count+1)])                  
    #print(volumes)
    #print(grids)
    #print(st)
    #print("volumes",volumes)
    #print(grids)
    #print(volumes)
    open_pos = pd.Series(np.zeros(count) , index = grids)
    current_open_pos = pd.Series([pd.Series({"state":0})]*count , index = grids)
    current_open_positions = []
    #close_pos = pd.Series(np.zeros(count) , index = grids)
    #print("distance",D)
    big = 0
    current_open_count = 0
    #print(grids[-1])
    start_state = find_state(btc.iloc[0],grids)
    end_state = find_state(btc.iloc[-1],grids)
    down_trend = 0
    uping_trend = 0
    max_down_trend = 0
    max_uping_trend = 0
    double_up_move = 0
    sl_close = 0
    tp_close = 0
    grid_count = []
    Opens = 0
    last_open_grid = 0
    for j in range(btc.shape[0]):
        candle = btc.iloc[j]
        gap = candle["High"] - candle["Low"]
        candle_gap.append(candle["High"] - candle["Low"])
        state = check_state(candle,grids)
        if(D < gap):
            big += 1
        if(Opens>0):
                mean_open = 0
                invest = 0
                Vol = 0
                for X in current_open_positions:
                    Vol += X["volume"] 
                    mean_open += X["open_price"]*X["volume"]
                    invest += X["invest"]
                mean_open = mean_open/Vol
                if((current_open_positions[-1]["state"]=="b")and(mean_open*(1+tp)>candle["Low"])and(mean_open*(1+tp)<candle["High"])):
                    close = {}
                    close["close"] = "tp" 
                    close["state"] = "b"
                    close["open_avrg"] = mean_open
                    close["close_date"] = candle.name
                    close["close_avrg"] = mean_open*(1+tp)
                    close["volume"] = Vol
                    close["count"] = Opens
                    
                    current_open_positions = []
                    trade_details.append(close)
                    net_money += (close["close_avrg"]*close["volume"])*(1-t_cost)
                    tp_close+=Opens
                    Opens = 0
                    last_open_grid = 0
                    continue
                elif((current_open_positions[-1]["state"]=="s")and(mean_open*(1-tp)<candle["High"])):
                    close = {}
                    close["close"] = "tp"
                    close["state"] = "s"
                    close["open_avrg"] = mean_open
                    close["close_date"] = candle.name
                    close["close_avrg"] = mean_open*(1-tp)
                    close["volume"] = Vol
                    close["count"] = Opens
                    tp_close += Opens
                    trade_details.append(close)
                    Opens = 0
                    net_money += (invest +(close["volume"]*close["open_avrg"]-close["volume"]*close["close_avrg"]))*(1-t_cost)
                    current_open_positions = []
                    last_open_grid = 0
                    continue   
        if(state==-1):
            continue
        else:
            #try:
            for Grid in state:
                grid_count.append(len(state))
                last_grid = current_grid
                current_grid= Grid
                if(Opens==0):
                    if(candle["Close"]>candle["Open"]):
                        current_open_positions.append({"grid":Grid,"state":"s","invest" : set_invest(net_money,Opens,volumes),"open_price":grids[Grid],"open_date":candle.name,"volume":set_invest(net_money,Opens,volumes)/grids[Grid],"net_money":net_money,"pos_count":Opens+1})
                        net_money -= set_invest(net_money,Opens,volumes)
                        last_open_grid = Grid
                        Opens += 1
                        #print(datetime.datetime.now())
                    elif(candle["Open"]>candle["Close"]):
                        current_open_positions.append({"grid":Grid,"state":"b","invest": set_invest(net_money,Opens,volumes) ,"open_price":grids[Grid],"open_date":candle.name,"volume":set_invest(net_money,Opens,volumes)/grids[Grid],"net_money":net_money,"pos_count":Opens+1})
                        net_money -= set_invest(net_money,Opens,volumes)
                        last_open_grid = Grid
                        Opens += 1
                        #print(datetime.datetime.now())                
                elif(last_open_grid<current_grid):
                    #if(Opens != 0):
                        #for Sl 
                        #first_pos = current_open_positions[0]
                        mean_open = 0
                        Vol = 0
                        invest = 0
                        for X in current_open_positions:
                            Vol += X["volume"]
                            mean_open += X["open_price"]*X["volume"]
                            invest+= X["invest"]
                            
                        mean_open = mean_open/Vol
                        if(((candle["High"]/mean_open)-1>sl)and(X["state"]=="s")):
                                close = {}
                                close["close"] = "sl"
                                close["state"] = "s"
                                close["open_avrg"] = mean_open
                                close["close_date"] = candle.name
                                close["close_avrg"] = mean_open*(1+sl)
                                close["volume"] = Vol
                                close["count"] = Opens
                                trade_details.append(close)
                                
                                net_money += (invest + (close["open_avrg"]*close["volume"]-close["close_avrg"]*close["volume"]))*(1-t_cost)
                                current_open_positions = []
                                sl_close += Opens
                                Opens = 0
                                last_open_grid = 0
                                continue 
                        last_pos = current_open_positions[-1]
                        if((last_pos["state"] == "s")and(candle["rsi"]>=100-rsi_limit)):
                            if((Opens<open_pos_count)and(j<btc.shape[0] - open_date_limit)):
                                current_open_positions.append({"grid":Grid,"state":"s","invest":set_invest(net_money,Opens,volumes),"open_price":grids[Grid],"open_date":candle.name,"volume":set_invest(net_money,Opens,volumes)/grids[Grid],"net_money":net_money,"pos_count":Opens+1})
                                net_money -= set_invest(net_money,Opens,volumes)
                                Opens+=1
                                last_open_grid = Grid
                                continue
                            elif(Opens==open_pos_count):
                            #something should do!!!
                                continue

                    # elif((len(current_open_positions)==0)and(candle["rsi"]>=100 - rsi_limit)and(j<btc.shape[0] - open_date_limit)):
                    #     current_open_positions.append({"state":"s","invest":net_money/(open_pos_count-Opens),"open_price":grids[Grid],"open_date":candle.name,"volume":net_money/((open_pos_count-Opens)*grids[Grid]),"net_money":net_money})
                    #     net_money -= (net_money)/(open_pos_count - Opens)
                    #     last_open_grid = grids[Grid]
                    #     Opens += 1   
                elif(last_open_grid>current_grid):
                    #if(Opens!=0):
                        #first_pos = current_open_positions[0]
                        mean_open = 0
                        Vol = 0
                        for X in current_open_positions:
                            mean_open += X["open_price"]*X["volume"]
                            Vol += X["volume"]
                        mean_open = mean_open/Vol
                        if(((candle["Low"]/mean_open)<1-sl)and(X["state"]=="b")):
                                close = {}
                                close["close"] = "sl"
                                close["state"] = "b"
                                close["open_avrg"] = mean_open
                                close["close_date"] = candle.name
                                close["close_avrg"] = mean_open*(1-sl)
                                close["volume"] = Vol
                                close["count"] = Opens
                                trade_details.append(close)
                                sl_close += Opens
                                Opens = 0
                                net_money += (close["close_avrg"]*close["volume"])*(1-t_cost)
                                current_open_positions = []
                                last_open_grid = 0
                                continue 
                        last_pos = current_open_positions[-1]
                        if((last_pos["state"] == "b")and(candle["rsi"]<=rsi_limit)and(j<btc.shape[0] - open_date_limit)and(current_grid<last_open_grid)):
                            if(Opens<open_pos_count):
                                current_open_positions.append({"grid":Grid,"state":"b","invest":set_invest(net_money,Opens,volumes),"open_price":grids[Grid],"open_date":candle.name,"volume":set_invest(net_money,Opens,volumes)/grids[Grid],"net_money":net_money,"pos_count":Opens+1})
                                net_money -= set_invest(net_money,Opens,volumes)
                                Opens+=1
                                last_open_grid = Grid
                                continue
                            elif(Opens==open_pos_count):
                                continue
                        

                    # elif((Opens==0)and(candle["rsi"]<=rsi_limit)and(j<btc.shape[0] - open_date_limit)):
                    #     current_open_positions.append({"state":"b","invest":net_money/(open_pos_count-Opens),"open_price":grids[Grid],"open_date":candle.name,"volume":net_money/(((open_pos_count-Opens)*grids[Grid])),"net_money":net_money})
                    #     net_money -= (net_money)/(open_pos_count - Opens)
                    #     last_open_grid = grids
                    #     Opens += 1
                if(current_grid<last_grid):
                    down_trend+=1
                    uping_trend = 0
                    if(max_down_trend<down_trend):
                        max_down_trend = down_trend
                elif(current_grid>last_grid):
                    down_trend = 0
                    uping_trend += 1
                    if(uping_trend>2):
                        double_up_move += 1
                    if(max_uping_trend<uping_trend):
                        max_uping_trend = uping_trend
            #except:
                #print(state)
    candle_gap = np.array(candle_gap)
    trade_details = pd.DataFrame(trade_details)
    try:
    #if(True):   
        #print(trade_details)
        buy_pos = trade_details[trade_details["state"] == "b"].copy()
        #print("buy_pos is:",buy_pos)
        sell_pos = trade_details[trade_details["state"] == "s"].copy()
        buy_pos["gain"] = buy_pos["volume"]*(buy_pos["close_avrg"] - buy_pos["open_avrg"])
        sell_pos["gain"] = sell_pos["volume"]*(sell_pos["open_avrg"] - sell_pos["close_avrg"])
        #trade_details["gain"] = trade_details["volume"]*(trade_details["sell"] - trade_details["buy"])/trade_details["buy"]
        #buy_pos["duration"] = buy_pos["close_date"] - buy_pos["open_date"]
        #sell_pos["duration"] = sell_pos["close_date"] - sell_pos["open_date"]
        ##trade_details["duration"] = trade_details["sell_date"] - trade_details["buy_date"]
    except:
        buy_pos = np.nan 
        sell_pos = np.nan
    #else:
        print("no position exist")
        pass
    opened_count = trade_details.shape[0] + len(current_open_positions)
    close_pos_count = trade_details.shape[0]
    try:
        pass
        #trade_details["duration"] = trade_details["close_date"] - trade_details["open_date"]
    except:
        pass
    if(print_details==1):
        print("numbers of candles that difference is higher than distance",big)
        print("candle_gap mean" , candle_gap.mean() , "maximum candle_gap",candle_gap.max(),"candle_count",btc.shape[0])
        #print("lowest line:",grids[0],"highest line:",grids[-1])
        #print("lowest_low:",np.min(btc["Low"]),"highest_high:",np.max(btc["High"]))
        print("count of opened_position:",opened_count)
        print("close_pos:",close_pos_count)
        print("current_open_pos:" , len(current_open_positions))
        print(f"bellow {rsi_limit}",len(BTC[BTC["rsi"]<rsi_limit]) , f"upper {100-rsi_limit}",len(BTC[BTC["rsi"]>100 - rsi_limit]))
        print("mean of grids count between candle is:",np.mean(grid_count))
        try:
            print(f"bellow {rsi_limit}:",len(BTC[BTC["rsi"]<rsi_limit])/len(BTC),"upper 70:",len(BTC[BTC["rsi"]>70])/len(BTC))
        except:
            pass
    net_volume = 0
    buy_cost = 0
    sell_gain = 0
    kind = -1
    for i in current_open_positions:
        #print(grids[i],'  ',volumes.iloc[i],current_open_pos.iloc[i][1])
        if(i["state"]=="b"):
            net_volume += i["volume"]
            kind = 0
            #buy_cost += i["volume"] * i["open_price"]
        elif(i["state"]=="s"):
            net_volume += +i["volume"] 
            sell_gain += i["invest"]
            kind = 1
    if(True):
    #try:    
        net_money      
        net_coin = net_volume*btc.iloc[-1]["Close"]
        if(kind == 0 ):
            print("state: b")
            print("volume:",net_volume)
            net_asset = net_money + net_coin
            print("net_money:",net_asset,"   open_pos_count",len(current_open_positions))
        #total_gain = np.sum(sell_pos["gain"]) + np.sum(buy_pos["gain"])
        elif(kind == 1):
            print("state: s")
            print("net_volume:",net_volume)
            net_asset = net_money + sell_gain + (sell_gain/btc.iloc[-1]["Close"] - net_volume) * btc.iloc[-1]["Close"]
            print("net_money:",net_asset,"   open_pos_count",len(current_open_positions))
        #print("open_pos_count is:",open_pos_count)
        else:
            net_asset = net_money
            print("net_money:",net_money,"   open_pos_count",len(current_open_positions))
        if(print_details==1):
            print("sl_close count is: ",sl_close,"   tp_close count is:",tp_close)
            print("net_money_hold:",net_money,"net cion value is:",net_coin) 
            print("total:",net_asset)
            #print("\n","total gain is:",total_gain,"\n")
            print("close_price",btc["Close"].iloc[1])
            print("buy with first day open and sell with the last day close price, asset would be:",(btc["Close"].iloc[-1]/btc["Open"].iloc[0])*first_invest,"\n")
            #print("if buy with the lowest price of first day sell with the highest price of last day :",(close_price/open_price)*first_invest, "\n")
        #print("highest profit can gain with buy and hold is:",(np.max(btc["High"])/np.min(btc["Low"]))*total_invest - total_invest)
        details = {"start-data":start,"opened_count":opened_count,"closed_count":close_pos_count,"total":net_asset,"hold":(btc["Close"].iloc[-1]/btc["Open"].iloc[0])*first_invest,"sl_count":sl_close}
        market_state = {"start_state":start_state,"end_state":end_state,"max_down_trend":max_down_trend,"max_uping_trend":max_uping_trend,"triple_up_moving":double_up_move}
    #except:
    else:
        details = {"opened_count":opened_count,"closed_count":close_pos_count}
        return{"grids":grids,"current_open_pos":current_open_pos,"close_pos":close_pos,"opened_pos":open_pos,"trades_df":pd.DataFrame(trade_details),"candle_gap":candle_gap,"details":details,"rsi":btc["rsi"]} 
    return{"grids":grids,"current_open_pos":current_open_pos,"trades_df":pd.DataFrame(trade_details),"candle_gap":candle_gap,"details":details,"start_state":start_state,"end_state":end_state,"max_down_tren":max_down_trend,"volumes":volume,"market_state":market_state,"rsi":btc["rsi"],"buy_pos":buy_pos,"sell_pos":sell_pos,"open_positions":current_open_positions}

In [4]:
def show_chart(data,grids,result):
    trade_df = result["trades_df"]
    buy_pos = trade_df[trade_df["state"]=="b"][["open_price","open_date","close_price","close_date"]]
    sell_pos = trade_df[trade_df["state"]=="s"][["open_price","close_price","close_date","open_date"]]
    dates = data
# Assuming your data is similar to the mplfinance example
    max_pr = np.max(dates["High"])
    min_pr = np.min(dates["Low"])
    grids = [element for element in grids if min_pr <= element <= max_pr]
    #print(len(grids))
    #grids.append(2*grids[-1]-grids[-2])
    opens = dates["Open"]
    highs = dates["High"]
    lows = dates["Low"]
    closes = dates["Close"]
    Dates = dates.index

    trace = go.Candlestick(x=Dates, open=opens, high=highs, low=lows, close=closes)
    shapes = []
    for i in grids: 
# Define horizontal line shape
    
        hline_shape = go.layout.Shape(
        type='line',
        x0=min(Dates),  # Extend line from minimum date
        x1=max(Dates),  # Extend line to maximum date
        y0=i,  # Set y-axis position (replace with your desired value)
        y1=i,
        line=dict(color='red',width = .2),  # Set line style
        name='Resistance Level',
                  # Optional label for the line
        )
        shapes.append(hline_shape)
# Create layout and add shapes
    points1 = go.Scatter(x=buy_pos["open_date"], y=buy_pos["open_price"],mode = "markers",marker = dict(color = "blue",size = 10,symbol = "circle"))
    points2 = go.Scatter(x=buy_pos["close_date"], y=buy_pos["close_price"],mode = "markers",marker = dict(color = "red",size = 10,symbol = "circle"))
    points3 = go.Scatter(x=sell_pos["open_date"], y=sell_pos["open_price"],mode = "markers",marker = dict(color = "orange",size = 10,symbol = "circle"))
    points4 = go.Scatter(x=sell_pos["close_date"], y=sell_pos["close_price"],mode = "markers",marker = dict(color = "purple",size = 10,symbol = "circle"))
    layout = go.Layout(xaxis_title='Date', yaxis_title='Price',width = 2000,height = 800,yaxis_range = (2*grids[0]-grids[1],2*grids[-1]-grids[-2]))
    layout.update(shapes=shapes)
    #layout.update(shapes = trace)
    #layout.update(shapes = [points1,points2,points3,points4],layout = layout)
    tickvals = grids  # Example values (adjust as needed)
    ticktext = [str(i) for i in grids]  # Corresponding labels
    fig = go.Figure(data= [trace,points1,points2,points3,points4], layout=layout)
    layout.update(yaxis=dict(tickvals=tickvals, ticktext=ticktext))
# Create figure and show
    #fig = go.Figure(data=[trace], layout=layout)
    fig.show()
    fig.write_html("my_candlestick_chart.html")

In [5]:
def set_invest(invest,opens,volumes):
    return invest*volumes[opens] / sum(volumes[opens:])


eurusd = pd.read_pickle("EURUSD_all.pickle")
eurusd.columns = ["Open","High","Low","Close","Datetime"]
eurusd.index = eurusd["Datetime"]
eurgbp = pd.read_pickle("EURGBP.pickle")
eurgbp.columns = ["Open","High","Low","Close","Datetime"]
eurgbp.index = eurgbp["Datetime"]
eurusd2024 = pd.read_pickle("EURUSD_2024.pickle")
eurusd2024.columns = ["Open","High","Low","Close","Datetime"]
eurusd2024.index = eurusd2024["Datetime"]

In [6]:
btc2024 = pd.read_pickle("btc2024.pickle")
eth2024 = pd.read_pickle("ethusd2024.pickle")


In [7]:
btc2024.index = btc2024["Time"]
btc2024.columns = ["Datetime","Open","Close","High","Low"]
eth2024.index = eth2024['Time']
eth2024.columns = ["Datetime","Open","Close","High","Low"]

In [29]:
start = datetime.datetime(2023,1,1)
end  = datetime.datetime(2023,12,1)
result = count_grid(t_cost = .006,market = 'btc-usd',volumes = [1,1,1],start = start,end = end,
           gap =300,open_pos_count = 3,tp = .05,sl = .05,open_date_limit = 5*12,data = btc2024,interval = "60m" ,download = True,total_invest = 5000,print_details = 1,rsi_limit = 30,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
df = result["trades_df"]
try:
    print("win rate:",df[df["close"]=="tp"].shape[0]/df.shape[0])
    print("mean of:",df["count"].mean())
    print("max count",(df["count"]==3).sum())
except:
    pass

result["trades_df"]


#show_chart(eurgbp.loc[start:end],result["grids"],result)

[*********************100%%**********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'rsi'], dtype='object')
(10885, 7)
Open         31803.693359
High         31957.285156
Low          31730.138672
Close        31950.976562
Adj Close    31950.976562
Volume           0.000000
rsi                   NaN
Name: 2022-06-01 00:00:00, dtype: float64


numbers of candles that difference is higher than distance 718
candle_gap mean 120.91935271445797 maximum candle_gap 2274.126953125 candle_count 10885
count of opened_position: 139
close_pos: 139
current_open_pos: 0
bellow 30 1436 upper 70 1390
mean of grids count between candle is: 1.2174154187465336
bellow 30: 0.13192466697289848 upper 70: 0.12769866789159393
net_money: 6736.752858057367    open_pos_count 0
sl_close count is:  93    tp_close count is: 144
net_money_hold: 6736.752858057367 net cion value is: 0.0
total: 6736.752858057367
close_price 31899.189453125
buy with first day open and sell with the last day close price, asset would be: 4076.733300003052 

Empty DataFrame
Columns: []
Index: []
win rate: 0.7266187050359713
mean of: 1.7050359712230216
max count 41


,close,state,open_avrg,close_date,close_avrg,volume,count
0,sl,b,31415.454428,2022-06-01 20:00:00,29844.681706,0.159157,3
1,tp,b,29617.364258,2022-06-06 03:00:00,31098.232471,0.053139,1
2,tp,s,31417.364258,2022-06-06 09:00:00,29846.496045,0.050824,1
3,tp,s,31417.364258,2022-06-06 11:00:00,29846.496045,0.051564,1
4,tp,s,31417.364258,2022-06-06 13:00:00,29846.496045,0.052316,1
...,...,...,...,...,...,...,...
134,sl,b,27517.364258,2023-08-17 21:00:00,26141.496045,0.081524,1
135,sl,b,26917.364258,2023-08-17 21:00:00,25571.496045,0.081794,1
136,tp,b,26015.058003,2023-08-29 14:00:00,27315.810904,0.249177,3
137,sl,b,27517.364258,2023-08-31 20:00:00,26141.496045,0.081956,1


In [49]:
eurgbp

,Open,High,Low,Close,Datetime
Datetime,,,,,
2020-11-17 00:00:00,0.89793,0.89837,0.89793,0.89829,2020-11-17 00:00:00
2020-11-17 00:05:00,0.89829,0.89830,0.89789,0.89820,2020-11-17 00:05:00
2020-11-17 00:10:00,0.89797,0.89809,0.89744,0.89796,2020-11-17 00:10:00
2020-11-17 00:15:00,0.89795,0.89828,0.89788,0.89828,2020-11-17 00:15:00
2020-11-17 00:20:00,0.89828,0.89835,0.89787,0.89803,2020-11-17 00:20:00
...,...,...,...,...,...
2023-07-19 12:30:00,0.86768,0.86781,0.86755,0.86779,2023-07-19 12:30:00
2023-07-19 12:35:00,0.86779,0.86780,0.86750,0.86765,2023-07-19 12:35:00
2023-07-19 12:40:00,0.86765,0.86772,0.86727,0.86748,2023-07-19 12:40:00


In [55]:
start = datetime.datetime(2023,1,1)
end  = datetime.datetime(2023,12,1)
result = count_grid(t_cost = 0,market = 'BTC-usd',volumes = [1,2,1],start = start,end = end,
           gap =.0008,open_pos_count = 3,tp = .0015,sl = .0015,open_date_limit = 5*12,data = eurgbp,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = 100,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
df = result["trades_df"]
try:
    print("win rate:",df[df["close"]=="tp"].shape[0]/df.shape[0])
    print("mean of:",df["count"].mean())
except:
    pass

result["trades_df"]

<class 'pandas.core.frame.DataFrame'>
Index(['Open', 'High', 'Low', 'Close', 'Datetime', 'rsi'], dtype='object')
(40514, 6)
Open                    0.88532
High                    0.88557
Low                     0.88532
Close                   0.88557
Datetime    2023-01-02 07:00:00
rsi                         NaN
Name: 2023-01-02 07:00:00, dtype: object


C:\Users\Saman\AppData\Local\Temp\ipykernel_24208\3881006277.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc["rsi"] = rsi(btc["Close"])


numbers of candles that difference is higher than distance 705
candle_gap mean 0.00026356864293824365 maximum candle_gap 0.0042999999999999705 candle_count 40514
count of opened_position: 974
close_pos: 973
current_open_pos: 1
bellow 100 40501 upper 0 40501
mean of grids count between candle is: 1.0343148990503552
bellow 100: 0.9996791232660315 upper 70: 0.09643579997038061
state: b
volume: 1738.7930857992365
net_money: 6034.968266330274    open_pos_count 1
sl_close count is:  444    tp_close count is: 963
net_money_hold: 4527.086902325176 net cion value is: 1507.8813640050978
total: 6034.968266330274
close_price 0.88521
buy with first day open and sell with the last day close price, asset would be: 4897.664121447612 

   grid state       invest  open_price           open_date       volume  \
0    20     b  1509.028967     0.86786 2023-07-19 12:20:00  1738.793086   

    net_money  pos_count  
0  6036.11587          1  
win rate: 0.8108941418293937
mean of: 1.4460431654676258


,close,state,open_avrg,close_date,close_avrg,volume,count
0,tp,s,0.88546,2023-01-02 07:05:00,0.884132,1411.695616,1
1,tp,b,0.88466,2023-01-02 10:55:00,0.885987,5654.010631,3
2,tp,s,0.88706,2023-01-02 11:05:00,0.885729,1411.792272,1
3,sl,b,0.88546,2023-01-03 01:10:00,0.884131,5659.497182,3
4,sl,b,0.88466,2023-01-03 01:15:00,0.883333,1414.028962,1
...,...,...,...,...,...,...,...
968,tp,s,0.86546,2023-07-19 09:55:00,0.864162,1740.349315,1
969,tp,s,0.86546,2023-07-19 10:10:00,0.864162,1741.001946,1
970,tp,s,0.86706,2023-07-19 10:20:00,0.865759,1738.440918,1
971,tp,s,0.86706,2023-07-19 10:40:00,0.865759,1739.092833,1


In [230]:
btc2024.iloc[-1]

Datetime    2024-05-27 10:15:00
Open                   68575.33
Close                  68575.33
High                   68575.33
Low                    68575.33
Name: 2024-05-27 10:15:00, dtype: object

In [152]:
data = yf.download("BTC-usd",start = datetime.datetime(2023,1,1),end = datetime.datetime(2023,2,1),interval = "5m")

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: Exception('%ticker%: 5m data not available for startTime=1672531200 and endTime=1675209600. The requested range must be within the last 60 days.')


In [145]:
data

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-05-01 00:00:00+00:00,60612.921875,60698.964844,60612.921875,60645.257812,60645.257812,0
2024-05-01 00:05:00+00:00,60634.800781,60734.496094,60634.800781,60734.496094,60734.496094,0
2024-05-01 00:10:00+00:00,60708.449219,60763.929688,60651.843750,60651.843750,60651.843750,29454336
2024-05-01 00:15:00+00:00,60613.691406,60644.644531,60549.027344,60549.027344,60549.027344,9850880
2024-05-01 00:20:00+00:00,60499.359375,60499.359375,60437.320312,60483.929688,60483.929688,43159552
...,...,...,...,...,...,...
2024-05-01 23:35:00+00:00,57963.429688,58128.449219,57957.500000,58128.449219,58128.449219,0
2024-05-01 23:40:00+00:00,58142.300781,58287.421875,58142.300781,58264.476562,58264.476562,166506496
2024-05-01 23:45:00+00:00,58282.460938,58407.632812,58282.460938,58327.472656,58327.472656,79233024


In [147]:
btc2024.loc[datetime.datetime(2024,5,1):datetime.datetime(2024,5,2)]

,Datetime,Open,Close,High,Low
Time,,,,,
2024-05-01 00:00:00,2024-05-01 00:00:00,59963.59,59869.88,59963.59,59869.88
2024-05-01 00:05:00,2024-05-01 00:05:00,59869.88,59800.00,59869.89,59800.00
2024-05-01 00:10:00,2024-05-01 00:10:00,59800.00,59963.58,59963.58,59800.00
2024-05-01 00:15:00,2024-05-01 00:15:00,59963.58,60109.35,60109.35,59963.58
2024-05-01 00:20:00,2024-05-01 00:20:00,60109.34,60109.34,60109.34,59999.00
...,...,...,...,...,...
2024-05-01 23:40:00,2024-05-01 23:40:00,57295.78,57295.78,57295.78,57295.78
2024-05-01 23:45:00,2024-05-01 23:45:00,57295.78,56973.90,57295.78,56965.20
2024-05-01 23:50:00,2024-05-01 23:50:00,56973.90,56973.90,56973.90,56973.90


In [135]:
x = [1, 2, 3, 4, 5]
y = [3, 5, 7, 2, 1]

# Color list (modify colors as needed)
colors = ['blue', 'green', 'red', 'orange', 'purple']

# Create scatter trace with colored points (no line)
trace = go.Scatter(
    x=x,
    y=y,
    mode='markers',  # Set mode to 'markers' for points only
    marker=dict(color=colors, size=10)  # Adjust marker size as needed
)

# Create layout and figure
layout = go.Layout(xaxis_title='X-axis', yaxis_title='Y-axis', title='Colored Points (Markers Only)')
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [79]:
b = np.array([1,2,3,4,5])
b[2.1:3.1]

TypeError: slice indices must be integers or None or have an __index__ method

In [108]:
data = eurusd2024.loc[datetime.datetime(2024,3,1):datetime.datetime(2024,4,1)]

In [67]:
print(np.max(data["High"]))
print(np.min(data["Low"]))


1.10451
1.07945


In [ ]:
#def do_grid(data,gap,grid_count,sl,rsi,open_limition)
result = count_grid('BTC-usd',start = datetime.datetime(2022,6,1),end = datetime.datetime(2024,1,1),
           gap = 2000,open_pos_count = 3,sl = .1,open_date_limit = 5*12,interval = "1h" ,download = True ,total_invest = 5000,print_details = 1,rsi_limit = 30,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
result["trades_df"]

In [16]:
#indeices = eurusd.loc[datetime.datetime(2022,6,1):datetime.datetime(2024,4,1)].index

In [75]:
#data = pd.read_pickle("eurusd.pickle")
eurusd = pd.read_pickle("EURUSD.pickle")
eurusd.columns = ["Open","High","Low","Close","Datetime"]
eurusd.index = eurusd["Datetime"]
eurgbp = pd.read_pickle("EURGBP.pickle")
eurgbp.columns = ["Open","High","Low","Close","Datetime"]
eurgbp.index = eurgbp["Datetime"]
eurusd2024 = pd.read_pickle("EURUSD_2024.pickle")
eurusd2024.columns = ["Open","High","Low","Close","Datetime"]
eurusd2024.index = eurusd2024["Datetime"]


In [9]:
eurgbp.index

DatetimeIndex(['2020-11-17 00:00:00', '2020-11-17 00:05:00',
               '2020-11-17 00:10:00', '2020-11-17 00:15:00',
               '2020-11-17 00:20:00', '2020-11-17 00:25:00',
               '2020-11-17 00:30:00', '2020-11-17 00:35:00',
               '2020-11-17 00:40:00', '2020-11-17 00:45:00',
               ...
               '2023-07-19 12:05:00', '2023-07-19 12:10:00',
               '2023-07-19 12:15:00', '2023-07-19 12:20:00',
               '2023-07-19 12:25:00', '2023-07-19 12:30:00',
               '2023-07-19 12:35:00', '2023-07-19 12:40:00',
               '2023-07-19 12:45:00', '2023-07-19 12:50:00'],
              dtype='datetime64[ns]', name='Datetime', length=198704, freq=None)

In [ ]:
((data["high"] - data["low"])/data["low"]).mean()

In [17]:
eurusd2024

,Open,High,Low,Close,Datetime
Datetime,,,,,
2024-01-02 00:00:00,1.10437,1.10437,1.10415,1.10420,2024-01-02 00:00:00
2024-01-02 00:05:00,1.10420,1.10451,1.10419,1.10420,2024-01-02 00:05:00
2024-01-02 00:10:00,1.10420,1.10450,1.10420,1.10449,2024-01-02 00:10:00
2024-01-02 00:15:00,1.10420,1.10450,1.10420,1.10420,2024-01-02 00:15:00
2024-01-02 00:20:00,1.10427,1.10448,1.10420,1.10425,2024-01-02 00:20:00
...,...,...,...,...,...
2024-04-30 09:25:00,1.06929,1.06944,1.06919,1.06943,2024-04-30 09:25:00
2024-04-30 09:30:00,1.06940,1.06946,1.06922,1.06932,2024-04-30 09:30:00
2024-04-30 09:35:00,1.06932,1.06954,1.06902,1.06953,2024-04-30 09:35:00


In [69]:
result = count_grid('eurusd',start = datetime.datetime(2024,4,1),end = datetime.datetime(2024,5,1),
           gap = .002,open_pos_count = 4,sl = .005,open_date_limit = 5*12,data = eurusd2024,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = 100,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
result["trades_df"]

<class 'pandas.core.frame.DataFrame'>
Index(['Open', 'High', 'Low', 'Close', 'Datetime', 'rsi'], dtype='object')
(6166, 6)


C:\Users\amirs\AppData\Local\Temp\ipykernel_7048\3135045245.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc["rsi"] = rsi(btc["Close"])


numbers of candles that difference is higher than distance 10
candle_gap mean 0.00027795977943561174 maximum candle_gap 0.00486999999999993 candle_count 6166
count of opened_position: 41
close_pos: 39
current_open_pos: 2
bellow 100 6153 upper 0 6153
mean of grids count between candle is: 1.0183276059564719
bellow 100: 0.9978916639636718 upper 70: 0.1300681154719429
net_money: 5036.294364370059    open_pos_count 2
sl_close count is:  4
net_money_hold: 3629.5380686213707 net cion value is: 1410.5233534950735
total: 5036.294364370059
close_price 1.07891
buy with first day open and sell with the last day close price, asset would be: 4960.2762584592565 

   grid state      invest  open_price           open_date      volume  \
0    27     s  502.270738    1.067415 2024-04-23 14:55:00  470.548698   
1    28     s  906.369087    1.069415 2024-04-26 19:35:00  847.537286   

     net_money  pos_count  
0  5022.707379          1  
1  4531.845433          2  


,grid,state,invest,open_price,open_date,volume,net_money,pos_count,close_price,close_date,close_pos_count,margin,sl_close,duration
0,33,s,500.000000,1.079415,2024-04-01 01:00:00,463.213870,5000.000000,1,1.077415,2024-04-01 16:45:00,1.0,0.926428,NaN,0 days 15:45:00
1,30,b,900.166757,1.073415,2024-04-01 18:35:00,838.600874,4500.833785,2,1.075415,2024-04-02 15:00:00,2.0,1.677202,NaN,0 days 20:25:00
2,31,b,500.092643,1.075415,2024-04-01 17:00:00,465.022938,5000.926428,1,1.077415,2024-04-02 17:00:00,1.0,0.930046,NaN,1 days 00:00:00
3,36,s,1008.712389,1.085415,2024-04-04 10:25:00,929.333378,2521.780972,4,1.083415,2024-04-04 22:40:00,4.0,1.858667,NaN,0 days 12:15:00
4,35,s,1080.763274,1.083415,2024-04-03 19:30:00,997.552437,3602.544246,3,1.081415,2024-04-05 15:30:00,3.0,1.995105,NaN,1 days 20:00:00
5,34,s,900.636062,1.081415,2024-04-03 17:00:00,832.831116,4503.180308,2,1.079415,2024-04-05 16:10:00,2.0,1.665662,NaN,1 days 23:10:00
6,36,s,1009.948742,1.085415,2024-04-08 17:30:00,930.472439,2524.871855,4,1.083415,2024-04-10 15:30:00,4.0,1.860945,NaN,1 days 22:00:00
7,35,s,1082.087938,1.083415,2024-04-05 18:35:00,998.775112,3606.959793,3,1.081415,2024-04-10 15:30:00,3.0,1.997550,NaN,4 days 20:55:00
8,34,s,901.739948,1.081415,2024-04-05 17:15:00,833.851896,4508.699742,2,1.079415,2024-04-10 15:30:00,2.0,1.667704,NaN,4 days 22:15:00
9,33,s,500.353368,1.079415,2024-04-03 17:00:00,463.541240,5003.533675,1,1.077415,2024-04-10 16:00:00,1.0,0.927082,NaN,6 days 23:00:00


In [398]:
result["trades_df"][["pos_count","close_pos_count"]].mean()

pos_count          1.615385
close_pos_count    1.615385
dtype: float64

In [393]:
def partial_grid(start,duration,n,gap,sl,rsi,open_limition,pos_count,data):
    results = []
    for i in range(n):
        try:
            result = result = count_grid('eurusd',start = start+i*datetime.timedelta(duration),end = start+(i+1)*datetime.timedelta(duration),
                gap = gap,open_pos_count = pos_count,sl = sl,open_date_limit = open_limition,data = data,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = rsi,compound_intrest=True)
            results.append(result["details"])
            
        except:
            pass
    results = pd.DataFrame(results)
    return results

In [ ]:
y = partial_grid(datetime.datetime(2020,12,1),data = eurgbp,duration = 120,n = 25,gap = .001,sl = .002,rsi = 100,open_limition = 12*10,pos_count = 10)

In [397]:
y

,start-data,opened_count,closed_count,total,hold,mean_duration,sl_count
0,2020-12-01,490,489,5027.371602,4764.860335,0 days 21:52:13.128834355,70
1,2021-03-31,378,370,5044.617652,4993.315194,0 days 16:38:41.351351351,43
2,2021-07-29,333,332,4972.421821,4935.953882,1 days 12:55:44.277108433,62
3,2021-11-26,515,513,4956.742127,4952.270566,0 days 22:48:43.976608187,98
4,2022-03-26,751,751,4992.201497,5105.672703,0 days 11:43:47.296937416,125
5,2022-07-24,1050,1049,5222.979476,5103.086724,0 days 10:16:50.962821734,109
6,2022-11-21,660,659,5080.344577,5032.331685,0 days 13:44:11.745068285,83
7,2023-03-21,332,332,4991.084730,4932.597267,1 days 11:28:01.626506024,56
8,2023-07-19,4,3,4998.334910,5036.999175,0 days 02:20:00,1


In [358]:
a = datetime.datetime(2023,1,1)
a+20*datetime.timedelta(4)

datetime.datetime(2023, 3, 22, 0, 0)

In [201]:
result["trades_df"][["open_date","state","open_price","close_price","pos_count","close_pos_count","close_date"]]

,open_date,state,open_price,close_price,pos_count,close_pos_count,close_date
0,2023-07-03 03:45:00,s,1.090335,1.088335,1,1,2023-07-03 10:05:00
1,2023-07-03 14:10:00,s,1.090335,1.088335,1,1,2023-07-04 22:35:00
2,2023-07-05 10:10:00,s,1.090335,1.088335,1,1,2023-07-05 11:00:00
3,2023-07-06 04:25:00,b,1.084335,1.086335,2,2,2023-07-06 09:20:00
4,2023-07-05 19:00:00,b,1.086335,1.088335,1,1,2023-07-06 13:25:00
5,2023-07-06 15:15:00,b,1.086335,1.088335,1,1,2023-07-06 16:00:00
6,2023-07-06 17:05:00,b,1.086335,1.088335,1,1,2023-07-06 19:05:00
7,2023-07-11 20:00:00,s,1.100335,1.098335,5,5,2023-07-27 21:25:00


In [ ]:
(data["High"]-data["Low"]).mean()

0.0004060239795709356

In [200]:
result["trades_df"].head(10)

,grid,state,invest,open_price,open_date,volume,net_money,pos_count,close_price,close_date,close_pos_count,margin,duration
0,28,s,1000.000000,1.090335,2023-07-03 03:45:00,917.149316,5000.000000,1,1.088335,2023-07-03 10:05:00,1,1.834299,0 days 06:20:00
1,28,s,1000.366860,1.090335,2023-07-03 14:10:00,917.485782,5001.834299,1,1.088335,2023-07-04 22:35:00,1,1.834972,1 days 08:25:00
2,28,s,1000.733854,1.090335,2023-07-05 10:10:00,917.822370,5003.669270,1,1.088335,2023-07-05 11:00:00,1,1.835645,0 days 00:50:00
3,25,b,1001.100983,1.084335,2023-07-06 04:25:00,923.239574,4004.403932,2,1.086335,2023-07-06 09:20:00,2,1.846479,0 days 04:55:00
4,26,b,1001.100983,1.086335,2023-07-05 19:00:00,921.539841,5005.504915,1,1.088335,2023-07-06 13:25:00,1,1.843080,0 days 18:25:00
5,26,b,1001.838895,1.086335,2023-07-06 15:15:00,922.219108,5009.194474,1,1.088335,2023-07-06 16:00:00,1,1.844438,0 days 00:45:00
6,26,b,1002.207782,1.086335,2023-07-06 17:05:00,922.558679,5011.038912,1,1.088335,2023-07-06 19:05:00,1,1.845117,0 days 02:00:00
7,33,s,1002.576806,1.100335,2023-07-11 20:00:00,911.155971,1002.576806,5,1.098335,2023-07-27 21:25:00,5,1.822312,16 days 01:25:00


In [ ]:
a = result["trades_df"]["close_price"].loc[11]
b = result["trades_df"]["open_price"].loc[11]
print(a-b)

0.0


In [319]:
result = count_grid('eurusd',start = datetime.datetime(2022,1,1),end = datetime.datetime(2022,2,1),
           gap = .001,open_pos_count = 3,sl = .002 ,open_date_limit = 300,data = eurgbp,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = 100,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
result["trades_df"]


<class 'pandas.core.frame.DataFrame'>
Index(['Open', 'High', 'Low', 'Close', 'Datetime', 'rsi'], dtype='object')
(6040, 6)


C:\Users\amirs\AppData\Local\Temp\ipykernel_12224\2367810808.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc["rsi"] = rsi(btc["Close"])


numbers of candles that difference is higher than distance 7
candle_gap mean 0.00024312086092715294 maximum candle_gap 0.0013100000000000334 candle_count 6040
count of opened_position: 66
close_pos: 65
current_open_pos: 1
bellow 100 6027 upper 0 6027
mean of grids count between candle is: 1.0013831258644537
bellow 100: 0.9978476821192053 upper 70: 0.09288079470198675
net_money: 5007.212017300811    open_pos_count 1
sl_close count is:  23
net_money_hold: 3338.0613855555066 net cion value is: 1668.9107538102028
total: 5007.212017300811
close_price 0.83962
buy with first day open and sell with the last day close price, asset would be: 4969.523809523809 

   grid state       invest  open_price           open_date       volume  \
0    28     s  1669.030693     0.83494 2022-01-31 17:55:00  1998.982793   

     net_money  pos_count  
0  5007.092078          1  


,grid,state,invest,open_price,open_date,volume,net_money,pos_count,close_price,close_date,margin,sl_close,close_pos_count,duration
0,32,s,1666.666667,0.83894,2022-01-03 00:05:00,1986.633927,5000.000000,1,0.840618,2022-01-03 01:35:00,-3.333333,True,1.0,0 days 01:30:00
1,32,b,1665.555556,0.83894,2022-01-03 10:30:00,1985.309504,3331.111111,2,0.839940,2022-01-03 11:50:00,1.985310,NaN,2.0,0 days 01:20:00
2,33,b,1665.555556,0.83994,2022-01-03 03:40:00,1982.945872,4996.666667,1,0.840940,2022-01-03 14:55:00,1.982946,NaN,1.0,0 days 11:15:00
3,33,b,1666.878307,0.83994,2022-01-03 16:55:00,1984.520689,5000.634922,1,0.840940,2022-01-03 17:35:00,1.984521,NaN,1.0,0 days 00:40:00
4,31,b,1667.539814,0.83794,2022-01-03 18:50:00,1990.046798,1667.539814,3,0.838940,2022-01-04 02:15:00,1.990047,NaN,3.0,0 days 07:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,26,b,1671.813609,0.83294,2022-01-27 16:35:00,2007.123693,3343.627218,2,0.831274,2022-01-28 16:10:00,-3.343627,True,NaN,0 days 23:35:00
61,25,b,1673.823145,0.83194,2022-01-28 11:55:00,2011.951757,1673.823145,3,0.831274,2022-01-28 16:10:00,-1.339718,True,NaN,0 days 04:15:00
62,26,s,1668.807643,0.83294,2022-01-28 19:45:00,2003.514830,3337.615286,2,0.831940,2022-01-28 23:45:00,2.003515,NaN,2.0,0 days 04:00:00
63,25,s,1668.807643,0.83194,2022-01-28 17:50:00,2005.923075,5006.422929,1,0.830940,2022-01-31 12:45:00,2.005923,NaN,1.0,2 days 18:55:00


In [247]:
a = result["trades_df"]
a = a[a["state"]=="s"]

In [207]:
(eurgbp["High"] - eurgbp["Low"]).mean()

0.0003025201304452852

In [ ]:
result = count_grid('eurusd',start = datetime.datetime(2023,10,1),end = datetime.datetime(2023,10,31),
           gap = .0008,open_pos_count = 10,sl = .01 ,open_date_limit = 300,data = data,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = 300,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
result["trades_df"]

In [ ]:
data.loc[datetime.datetime(2023,10,1):]

,Open,High,Low,Close,Datetime,rsi
Datetime,,,,,,
2023-10-02 00:15:00,1.05686,1.05717,1.05675,1.05675,2023-10-02 00:15:00,37.430168
2023-10-02 00:20:00,1.05695,1.05695,1.05659,1.05662,2023-10-02 00:20:00,32.065217
2023-10-02 00:25:00,1.05659,1.05676,1.05659,1.05675,2023-10-02 00:25:00,37.500000
2023-10-02 00:30:00,1.05676,1.05676,1.05660,1.05660,2023-10-02 00:30:00,35.467980
2023-10-02 00:35:00,1.05659,1.05666,1.05654,1.05663,2023-10-02 00:35:00,38.071066
...,...,...,...,...,...,...
2023-10-31 09:05:00,1.06099,1.06113,1.06062,1.06063,2023-10-31 09:05:00,68.372093
2023-10-31 09:10:00,1.06063,1.06066,1.06033,1.06039,2023-10-31 09:10:00,67.741935
2023-10-31 09:15:00,1.06038,1.06073,1.06038,1.06073,2023-10-31 09:15:00,72.111554


In [ ]:
((data["High"] - data["Low"])/data["Low"]).mean()

0.00032292574949974395

In [323]:
#market,gap,invest,ops,rsi_limit
def six_month_result(market,duration,gap,invest,ops,open_limit,rsi_limit,sl,interval = "1h",data=0):
    results = []
    dates = [
 datetime.datetime(2021,1,1,0,0),
 datetime.datetime(2021,2,1,0,0),
 datetime.datetime(2021,3,1,0,0),
 datetime.datetime(2021,4,1,0,0),
 datetime.datetime(2021,5,1,0,0),
 datetime.datetime(2021, 6, 1, 0, 0),
 datetime.datetime(2021, 7, 1, 0, 0),
 datetime.datetime(2021, 8, 1, 0, 0),
 datetime.datetime(2021, 9, 1, 0, 0),
 datetime.datetime(2021, 10, 1, 0, 0),
 datetime.datetime(2021, 11, 1, 0, 0),
 datetime.datetime(2021, 12, 1, 0, 0),
 datetime.datetime(2022,1,1,0,0),
 datetime.datetime(2022,2,1,0,0),
 datetime.datetime(2022,3,1,0,0),
 datetime.datetime(2022,4,1,0,0),
 datetime.datetime(2022,5,1,0,0),
 datetime.datetime(2022, 6, 1, 0, 0),
 datetime.datetime(2022, 7, 1, 0, 0),
 datetime.datetime(2022, 8, 1, 0, 0),
 datetime.datetime(2022, 9, 1, 0, 0),
 datetime.datetime(2022, 10, 1, 0, 0),
 datetime.datetime(2022, 11, 1, 0, 0),
 datetime.datetime(2022, 12, 1, 0, 0),
 datetime.datetime(2023, 1, 1, 0, 0),
 datetime.datetime(2023, 2, 1, 0, 0),
 datetime.datetime(2023, 3, 1, 0, 0),
 datetime.datetime(2023, 4, 1, 0, 0),
 datetime.datetime(2023, 5, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 7, 1, 0, 0),
 datetime.datetime(2023, 8, 1, 0, 0),
]
    #dates = [datetime.datetime(2022,6,1),datetime.datetime(2022,9,1),datetime.datetime(2022,12,1),datetime.datetime(2023,3,1),datetime.datetime(2023,6,1),datetime.datetime(2023,9,1),datetime.datetime(2023,12,1),datetime.datetime(2024,1,1)]
    for i in range(len(dates)-duration):
        result = count_grid(market,start = dates[i],end = dates[i+duration],
            gap = gap,open_pos_count = ops,sl = sl ,open_date_limit = open_limit,interval = interval ,download = False,data = data,rsi_limit = rsi_limit,total_invest = invest,compound_intrest = True)
        #result = count_grid('eurusd',start = datetime.datetime(2023,10,1),end = datetime.datetime(2023,10,31),
        #   gap = .0008,open_pos_count = 10,sl = .01 ,open_date_limit = 300,data = data,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = 300,compound_intrest=True)

        results.append(result["details"])
    results = pd.DataFrame(results)
    #result = result.drop(["net_money_hold","net_coin","opened_count"],axis = 1)
    return results


In [ ]:
df = six_month_result("eurgbp=X",duration = 1,gap = .001,invest = 5000,ops = 8,
rsi_limit = 100,open_limit = 300,sl = .005,interval = "5m",data = eurusd)

In [330]:
df

,start-data,opened_count,closed_count,total,hold,mean_duration
0,2021-01-01,124,123,2998.545006,2966.577361,0 days 13:06:52.195121951
1,2021-02-01,98,97,2964.534614,2938.192914,0 days 13:15:15.463917525
2,2021-03-01,119,117,3029.377736,2946.170011,0 days 14:35:17.948717948
3,2021-04-01,115,113,3012.884270,3067.938232,0 days 10:53:32.389380530
4,2021-05-01,75,75,3014.557607,2964.172523,0 days 14:43:20
5,2021-06-01,75,74,3032.037990,2989.568918,0 days 13:11:41.351351351
6,2021-07-01,95,91,3003.758223,2987.749387,0 days 14:58:04.615384615
7,2021-08-01,56,55,3012.753828,3019.383890,1 days 20:22:10.909090909
8,2021-09-01,80,79,3015.977294,3001.712508,0 days 19:37:35.696202531
9,2021-10-01,79,77,3003.313721,2948.868399,1 days 00:16:02.337662337


In [ ]:
data

,Open,High,Low,Close,Datetime
0,1.10522,1.10525,1.10512,1.10512,2023-07-26 01:55:00
1,1.10513,1.10524,1.10510,1.10518,2023-07-26 02:00:00
2,1.10518,1.10518,1.10509,1.10510,2023-07-26 02:05:00
3,1.10510,1.10522,1.10509,1.10520,2023-07-26 02:10:00
4,1.10521,1.10521,1.10508,1.10511,2023-07-26 02:15:00
...,...,...,...,...,...
19682,1.06099,1.06113,1.06062,1.06063,2023-10-31 09:05:00
19683,1.06063,1.06066,1.06033,1.06039,2023-10-31 09:10:00
19684,1.06038,1.06073,1.06038,1.06073,2023-10-31 09:15:00
19685,1.06073,1.06080,1.06055,1.06077,2023-10-31 09:20:00


In [ ]:
data.index = data["Datetime"]

In [ ]:
data

,Open,High,Low,Close,Datetime
Datetime,,,,,
2023-07-26 01:55:00,1.10522,1.10525,1.10512,1.10512,2023-07-26 01:55:00
2023-07-26 02:00:00,1.10513,1.10524,1.10510,1.10518,2023-07-26 02:00:00
2023-07-26 02:05:00,1.10518,1.10518,1.10509,1.10510,2023-07-26 02:05:00
2023-07-26 02:10:00,1.10510,1.10522,1.10509,1.10520,2023-07-26 02:10:00
2023-07-26 02:15:00,1.10521,1.10521,1.10508,1.10511,2023-07-26 02:15:00
...,...,...,...,...,...
2023-10-31 09:05:00,1.06099,1.06113,1.06062,1.06063,2023-10-31 09:05:00
2023-10-31 09:10:00,1.06063,1.06066,1.06033,1.06039,2023-10-31 09:10:00
2023-10-31 09:15:00,1.06038,1.06073,1.06038,1.06073,2023-10-31 09:15:00


In [ ]:
yf.download("eurusd=X",interval = "5m",start = datetime.datetime(2024,3,1))

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-01 00:00:00+00:00,1.080964,1.080964,1.080847,1.080847,1.080847,0
2024-03-01 00:05:00+00:00,1.080847,1.081081,1.080847,1.081081,1.081081,0
2024-03-01 00:10:00+00:00,1.081081,1.081198,1.081081,1.081198,1.081198,0
2024-03-01 00:15:00+00:00,1.081198,1.081198,1.081081,1.081081,1.081081,0
2024-03-01 00:20:00+00:00,1.081198,1.081198,1.081198,1.081198,1.081198,0
...,...,...,...,...,...,...
2024-04-29 07:30:00+01:00,1.072501,1.072501,1.072271,1.072271,1.072271,0
2024-04-29 07:35:00+01:00,1.072271,1.072386,1.072271,1.072271,1.072271,0
2024-04-29 07:40:00+01:00,1.072271,1.072386,1.072041,1.072041,1.072041,0


In [ ]:
result = count_grid('eurusd',start = datetime.datetime(2024,3,1),end = datetime.datetime(2024,3,29),
           gap = .0015,open_pos_count = 3,sl = .01 ,data = data,interval = "5m" ,download = False,total_invest = 5000,print_details = 1,rsi_limit = 100,compound_intrest=True)
print(pd.DataFrame(result["open_positions"]))
result["trades_df"]

<class 'pandas.core.frame.DataFrame'>
Index(['Open', 'High', 'Low', 'Close', 'Datetime', 'rsi'], dtype='object')
(0, 6)


IndexError: single positional indexer is out-of-bounds

In [ ]:
result = count_grid('eurgbp=X',start = datetime.datetime(2024,3,1),end = datetime.datetime(2024,4,1),
           gap = .003,open_pos_count = 3,sl = .01 ,interval = "5m" ,total_invest = 5000,print_details = 1,rsi_limit = 30,compound_intrest=True)

result["trades_df"]

[*********************100%%**********************]  1 of 1 completed

<class 'int'>
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'rsi'], dtype='object')


numbers of candles that difference is higher than distance 0
candle_gap mean 0.0001926599559410464 maximum candle_gap 0.0017499923706054688 candle_count 5939
lowest line: 0.7638000092506408 highest line: 0.9468000092506409
lowest_low: 0.8501399755477905 highest_high: 0.8601999878883362
count of opened_position: 4
close_pos: 4
current_open_pos: 0
bellow 30 577 upper 70 562
bellow 30: 0.09715440309816467 upper 70: 0.09462872537464219
total_invest 5000 buy_cost 0 sell_gain 0 open_pos_count 3
sl_close count is:  0
net_money_hold: 5000 net btc value is: 0.0
total: 5023.363462648122

 total gain is: 23.363462648121228 

buy with first day open and sell with the last day close price, asset would be: 4994.975191838107 



,state,open_price,open_date,volume,close_price,close_date,duration
0,s,0.8568,2024-03-01 14:40:00+00:00,1945.222512,0.8538,2024-03-05 15:00:00+00:00,4 days 00:20:00
1,b,0.8508,2024-03-08 13:30:00+00:00,1958.940584,0.8538,2024-03-12 07:00:00+00:00,3 days 17:30:00
2,s,0.8598,2024-03-22 10:10:00+00:00,1938.435274,0.8568,2024-03-25 10:55:00+00:00,3 days 00:45:00
3,s,0.8568,2024-03-21 12:00:00+00:00,1945.222512,0.8538,2024-03-28 21:20:00+00:00,7 days 09:20:00


In [ ]:
!pip install investpy

     ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.4 MB 1.3 MB/s eta 0:00:04
     ---------------------------------------- 0.0/4.4 MB 1.3 MB/s eta 0:00:04
      --------------------------------------- 0.1/4.4 MB 508.4 kB/s eta 0:00:09
      --------------------------------------- 0.1/4.4 MB 476.3 kB/s eta 0:00:10
     - -------------------------------------- 0.1/4.4 MB 450.6 kB/s eta 0:00:10
     - -------------------------------------- 0.2/4.4 MB 523.5 kB/s eta 0:00:09
     - -------------------------------------- 0.2/4.4 MB 565.6 kB/s eta 0:00:08
     -- ------------------------------------- 0.2/4.4 MB 602.4 kB/s eta 0:00:07
     -- ------------------------------------- 0.3/4.4 MB 610.3 kB/s eta 0:00:07
     --- ------------------------------------ 0.3/4.4 MB 635.0 kB/s eta 0:00:07
     --- ------------------------------------ 0.4/4.4 MB 636.6 kB/s eta 0:00:07
     --- ------------------------------------ 0.4/4.4

In [ ]:
import investpy

data = investpy.get_currency_cross_historical_data(
    currency_cross='EUR/USD',
    from_date='01/01/2020',
    to_date='01/01/2021',
    interval='5m'  # Set the interval to 5 minutes
)

print(data.head())


ValueError: ERR#0073: interval value should be a str type and it can just be either 'Daily', 'Weekly' or 'Monthly'.

In [ ]:
A = pd.read_csv("EURUSD_historical_data.csv")

In [ ]:
A.drop(["Unnamed: 7"],axis =1,inplace = False)

,Date,Open,High,Low,Close,Change(Pips),Change(%)
0,04/25/2024 00:00,1.07000,1.07040,1.0698,1.07024,2.4,0.02
1,04/24/2024 23:55,1.06986,1.07000,1.0697,1.06996,1.0,0.01
2,04/24/2024 23:50,1.06980,1.06990,1.0696,1.06985,0.5,0.00
3,04/24/2024 23:45,1.06977,1.06980,1.0696,1.06960,-1.7,-0.02
4,04/24/2024 23:40,1.06976,1.06980,1.0696,1.06980,0.4,0.00
...,...,...,...,...,...,...,...
896,04/19/2024 20:25,1.06530,1.06550,1.0652,1.06540,1.0,0.01
897,04/19/2024 20:20,1.06544,1.06550,1.0652,1.06540,-0.4,0.00
898,04/19/2024 20:15,1.06539,1.06547,1.0652,1.06544,0.5,0.00
899,04/19/2024 20:10,1.06540,1.06550,1.0652,1.06540,0.0,0.00


In [ ]:
A.drop(["Unnamed: 7"],axis = 1,inplace =True)

In [ ]:
a = A.iloc[0]

In [ ]:
type(a["High"])

numpy.float64

In [ ]:
a["Date"].astype('datetime64[s]')

AttributeError: 'str' object has no attribute 'astype'

In [ ]:
u = a["Date"]
print(u)

04/25/2024 00:00


In [ ]:
u[-2:]

'00'

In [ ]:
b = A["Date"].apply(lambda x:datetime.datetime(int(x[6:10]),int(x[:2]),int(x[3:5]),int(x[11:13]),int(x[-2:])))

In [ ]:
A = A.iloc[::-1]

In [ ]:
b.iloc[29]

Timestamp('2024-03-20 00:00:00')

In [ ]:
len(A)

901

In [ ]:
!pip install tvdatafeed

ERROR: Could not find a version that satisfies the requirement tvdatafeed (from versions: none)
ERROR: No matching distribution found for tvdatafeed


In [ ]:
# import requests

# # Set the Binance API URL (testnet for demonstration)
# client.API_URL = 'https://testnet.binance.vision/api'

# # Define parameters
# ticker = 'BTCUSDT'
# kline_interval = '5m'
# sinceThisDate = '1591258320000'  # Start timestamp (optional)
# untilThisDate = '1591258379999'  # End timestamp (optional)

# # Fetch historical klines
# candle = client.get_historical_klines(ticker, kline_interval, str(sinceThisDate), str(untilThisDate))

NameError: name 'client' is not defined

In [ ]:
import requests

def get_historical_candles(symbol, interval, limit):
    base_url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        candles = response.json()
        return candles
    else:
        print("Failed to fetch candles:", response.status_code)
        return None

# Example usage
symbol = "BTCUSDT"  # Symbol of the asset pair
interval = "5m"     # Interval of the candles (5 minutes)
limit = 100         # Number of candles to retrieve
historical_candles = get_historical_candles(symbol, interval, limit)

# Print the first few candles as an example
if historical_candles:
    for candle in historical_candles:
        print(candle)
else:
    print("No historical candles retrieved.")


ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=None)

In [94]:
import requests
url = "https://api.binance.com/api/v3/klines"
params = {
    "symbol": "BTCUSDT",
    "interval": "1h",  # 1-hour interval
    "limit": 100,      # Retrieve 100 Klines
}

response = requests.get(url, params=params)
klines = response.json()
klines.to_pickle("btc.piklce")

ConnectionError: HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v3/klines?symbol=BTCUSDT&interval=1h&limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019DC1EC3340>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [104]:
q = pd.DataFrame([],columns=["a","b","c"])
a = pd.DataFrame([[1,2,3]],columns=["a","b","c"])
b = pd.DataFrame([[4,5,6]],columns=["a","b","c"])
#b = pd.DataFrame([[8],[9],[10]],columns=["a","b","c"])


In [105]:
c = pd.concat([q,a,b],axis = 0)

In [107]:
c

,a,b,c
0,1,2,3
0,4,5,6
